In [4]:
!dpkg -l | grep libcudnn

ii  libcudnn8                       8.9.6.50-1+cuda12.2                     amd64        cuDNN runtime libraries


In [3]:
!docker -v

/usr/bin/sh: 1: docker: not found


In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Nov_22_10:17:15_PST_2023
Cuda compilation tools, release 12.3, V12.3.107
Build cuda_12.3.r12.3/compiler.33567101_0


In [16]:
!nvidia-smi

Mon Oct 14 17:09:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.157                Driver Version: 538.18       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX 3500 Ada Gene...    On  | 00000000:01:00.0 Off |                  Off |
| N/A   43C    P5              23W /  35W |      0MiB / 12282MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
!uname -m
!lsb_release -a

x86_64


No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


In [12]:
!export PATH=/opt/nvidia/hpc_sdk/Linux_x86_64/21.3/compilers/bin:$PATH
!export PATH=/opt/nvidia//hpc_sdk/Linux_x86_64/21.3/cuda/12.3/bin:$PATH

In [1]:
import pandas as pd
import numpy as np
import ast
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import gc

2024-10-16 16:14:17.185621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 16:14:17.348550: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 16:14:17.396845: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 16:14:17.710353: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print(tf.__version__)

2.17.0


In [3]:
gpus=tf.config.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)


I0000 00:00:1729095270.927927       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729095271.022583       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1729095271.022651       9 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [4]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
from tensorflow.keras.preprocessing.text import tokenizer_from_json

# Cargar el archivo JSON
with open('tokenizer.json') as json_file:
    tokenizer_json = json_file.read()

# Reconstruir el tokenizer desde el archivo JSON
tokenizer = tokenizer_from_json(tokenizer_json)

In [8]:
def create_conv1d():
    model = models.Sequential()
    model.add(layers.Input(shape=(2172, 1)))
    model.add(layers.Conv1D(64, kernel_size=5, activation='relu'))
    model.add(layers.Conv1D(64, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    return model

In [9]:
def create_lstm():
    model=models.Sequential()
    # Primera capa LSTM con return_sequences=True
    model.add(layers.LSTM(64, return_sequences=True))
    model.add(layers.Dropout(0.3)) # Añadir Dropout
    
    # Segunda capa LSTM con return_sequences=True
    model.add(layers.LSTM(128, return_sequences=True))
    model.add(layers.Dropout(0.3))

    return model

In [10]:
def create_cnn_lstm_model(num_classes):
    conv1d = create_conv1d()
    
    video_input = layers.Input(shape=(None, 2172, 1)) 
    # masked_input = layers.Masking(mask_value=0.0)(video_input)

    
    # Aplicar CNN a cada frame usando TimeDistributed
    cnn_features = layers.TimeDistributed(conv1d)(video_input)
    
    lstm=create_lstm()
    lstm_out= lstm(cnn_features)
    # Capa final de salida
    output = layers.Dense(num_classes, activation='linear')(lstm_out)
    
    model = models.Model(inputs=video_input, outputs=output)
    return model

In [11]:
%pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.8/792.8 kB 1.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(references, hypotheses):
    smoothing = SmoothingFunction().method4
    return np.mean([sentence_bleu([ref], hyp, smoothing_function=smoothing) for ref, hyp in zip(references, hypotheses)])

In [13]:
ctc_loss=tf.keras.losses.CTC()

In [14]:
def decode(predictions,greedy):
    predicted_classes = tf.argmax(predictions, axis=-1)
    mask = tf.not_equal(predicted_classes, 0)
    seq_lengths = tf.reduce_sum(tf.cast(mask, tf.int32), axis=-1)
    
    predictions=tf.transpose(predictions,[1,0,2])

    if greedy:
        decoded, _ = tf.nn.ctc_greedy_decoder(inputs=predictions, sequence_length=seq_lengths)
    
    else:
        decoded, _ = tf.nn.ctc_beam_search_decoder(inputs=predictions, sequence_length=seq_lengths)
    decoded_dense = tf.sparse.to_dense(decoded[0], default_value=-1).numpy()
    
    # Extraer las secuencias decodificadas eliminando los -1 (valores de relleno)
    decoded_sequences = []
    for seq in decoded_dense:
        decoded_sequences.append([val for val in seq if val != -1])
    
    return decoded_sequences

In [15]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [28]:
def length_fn(points, translations):
    # La longitud será la cantidad de keyframes (primera dimensión de los puntos)
    longitud=tf.shape(points)[0]
    return longitud


# limites = [300,500,700,1100]
    
# bucket_batch_sizes = [32, 32, 16, 8,8]

def bucketed_dataset(dataset):
    limites = [300,500,700,1100]
    
    bucket_batch_sizes = [1, 1, 1, 1, 1]
    return dataset.bucket_by_sequence_length(
            element_length_func=length_fn,       
            bucket_boundaries=limites, # Límites de los buckets
            bucket_batch_sizes=bucket_batch_sizes,  # Tamaños de batch para cada bucket
            padded_shapes=(
                # [None,48,48],
                [None, 2172],  
                [None]           
            ),
            padding_values=(
                tf.constant(0, dtype=tf.float32), # Relleno con ceros para los puntos 
                tf.constant(0, dtype=tf.int32) # Relleno con ceros para las traducciones 
            )
    )

In [29]:
with tf.device('/CPU:0'):
    train_dataset=tf.data.Dataset.load('./train').unbatch()
    val_dataset=tf.data.Dataset.load('./val').unbatch()
    test_dataset=tf.data.Dataset.load('./test').unbatch()
    # Aplicar bucketing y padding
    train_dataset= train_dataset.apply(bucketed_dataset)
    val_dataset= val_dataset.apply(bucketed_dataset)
    test_dataset= test_dataset.apply(bucketed_dataset)

In [18]:
!nvidia-smi

Wed Oct 16 16:15:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.157                Driver Version: 538.18       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX 3500 Ada Gene...    On  | 00000000:01:00.0 Off |                  Off |
| N/A   42C    P5              19W /  35W |    375MiB / 12282MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [19]:
train_dataset.cache('./cache_temporal')
test_dataset.cache('./cache_temporal')
val_dataset.cache('./cache_temporal')


<CacheDataset element_spec=(TensorSpec(shape=(None, None, 2172), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int32, name=None))>

In [20]:
train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 2172), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int32, name=None))>

In [21]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Rehabilita la GPU 0

In [33]:
!nvidia-smi

Mon Oct 14 17:14:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.157                Driver Version: 538.18       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX 3500 Ada Gene...    On  | 00000000:01:00.0 Off |                  Off |
| N/A   44C    P8               4W /  35W |    407MiB / 12282MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [22]:
from tqdm import tqdm
points_batch, translation_batch = next(iter(train_dataset))
points_batch.shape, translation_batch.shape

(TensorShape([2, 453, 2172]), TensorShape([2, 277]))

In [23]:
steps_per_epoch = train_dataset.cardinality().numpy()
validation_steps = val_dataset.cardinality().numpy()

In [24]:
# Entrenar el modelo usando un ciclo de entrenamiento personalizado
def training(model,optimizador,checkpoint_callback,early_stopping):
    epochs = 100
    model.compile(optimizer=optimizador, loss=tf.keras.losses.CTC(), metrics=[calculate_bleu])


# Iterar a través de los batches
    for epoch in range(epochs):
     with tf.device('/GPU:0'):
        with tqdm(total=steps_per_epoch, desc=f'Epoch {epoch+1}/{epochs}', unit='batch') as pbar:
            epoch_loss_avg = tf.keras.metrics.Mean()
            epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
            # Métricas para la validación
            val_loss_avg = tf.keras.metrics.Mean()
            val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
            val_bleu_avg = tf.keras.metrics.Mean()

            for epoch in tqdm(range(epochs), desc="Epoch"):
                epoch_loss_avg.reset_states()
                epoch_accuracy.reset_states()
                val_acc_metric.reset_states()
                val_loss_avg.reset_states()
                val_bleu_avg.reset_states()

                for step, (points_batch, translation_batch) in enumerate(train_dataset):
                    pbar.set_postfix({
                        "Loss": epoch_loss_avg.result().numpy(),
                        "Accuracy": epoch_accuracy.result().numpy()
                    })
                    
                    with tf.GradientTape() as tape:
                        predictions = model(points_batch, training=True)
                        loss = tf.reduce_mean(ctc_loss(translation_batch, predictions))
                        
                    # Calcular y aplicar gradientes
                    gradients = tape.gradient(loss, model.trainable_variables)
                    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    predictions_decoded=decode(predictions, greedy=True)
                    epoch_accuracy.update_state(translation_batch, predictions_decoded)
                    epoch_loss_avg.update_state(loss)
                    
                    
                
                
                with tqdm(total=validation_steps, desc=f'Epoch {epoch+1}/{epochs} (Validation)', unit='batch') as pbar_val:

                    for step, (points_batch, translation_batch) in enumerate(val_dataset):
                        
                        pbar_val.set_postfix({
                            "Val Loss": val_loss_avg.result().numpy(),
                            "Val Accuracy": val_acc_metric.result().numpy(),
                            "Val BLEU": val_bleu_avg.result().numpy()
                        })
                        
                        # Evaluar en el conjunto de validación al final de cada epoch usando BLEU
                        predictions = model(points_batch, training=False)

                        val_loss=tf.reduce_mean(ctc_loss(translation_batch,predictions))
                        val_loss_avg.update_state(val_loss)

                        val_predictions = decode(predictions, greedy=False)
                        val_hypotheses = [tokenizer.sequences_to_texts([pred]) for pred in val_predictions]


                        val_acc_metric.update_state(translation_batch,val_predictions)
                        val_references = [tokenizer.sequences_to_texts([ref.numpy()]) for ref in translation_batch]
                        val_hypotheses = [tokenizer.sequences_to_texts([pred]) for pred in np.argmax(val_predictions, axis=-1)]
                        val_bleu = calculate_bleu(references = val_references, hypotheses = val_hypotheses)
                        val_bleu_avg.update_state(val_bleu)
                    
                    

                checkpoint_callback.on_epoch_end(epoch, logs={'val_loss': val_loss_avg, 'val_accuracy': val_acc_metric, 'val_bleu': val_bleu_avg})

                if early_stopping.on_epoch_end(epoch, logs={'val_loss': val_loss_avg, 'val_accuracy': val_acc_metric, 'val_bleu': val_bleu_avg}):
                    print("Early stopping triggered")
                    break
                print(f"Epoch: {epoch}, Loss: {epoch_loss_avg}, Accuracy: {epoch_accuracy}")
                print(f'Epoch: {epoch}, val_loss: {val_loss_avg}, val_acc: {val_acc_metric}, val_bleu: {val_bleu_avg}')

            # Evaluar el modelo en el dataset de test
            #test_loss, test_acc = model.evaluate(test_dataset)
            #print(f'Test accuracy: {test_acc}')

In [30]:
num_classes=len(tokenizer.word_index) + 1
model = create_cnn_lstm_model(num_classes)

optimizador = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizador)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/best_model.keras',
    save_best_only=True,       # Guardar solo si es el mejor modelo hasta ahora
    monitor='val_loss',
    mode='min'  
)

early_stopping = tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)
training(model,optimizador,checkpoint_callback,early_stopping)

Epoch 1/100: 0batch [00:00, ?batch/s, Loss=0, Accuracy=0]W0000 00:00:1729095529.833107       9 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729095529.840534       9 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729095529.844522       9 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729095529.849770       9 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729095529.854826       9 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729095529.860183       9 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729095529.864114       9 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729095529.869807       9 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy

InvalidArgumentError: {{function_node __wrapped__ArgMax_device_/job:localhost/replica:0/task:0/device:GPU:0}} Reduction axis -1 is empty in shape [1,0] [Op:ArgMax] name: 